### Colab environment

In [ ]:
!git clone https://github.com/v3code/IGLU-simple-agent.git
%cd ./IGLU-simple-agent
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.python.sh | bash
!./initialize.sh --colab

In [ ]:
import sys
sys.path.insert(1,'/content/IGLU-simple-agent')

# Описание агентов
В начале данной работы предполагалось использование летающего агента по причине
того, что он обладает несколькими преимуществами:
- Непрерывное пространство действий, которое должно было позволить более точное и быстрое управление агентом
- Возможность передвигаться по трем осям одновременно
- Практически отсутствуют столкновение с блоками (можно проходить сквозь блоки)

Однако, на самом деле, существуют некоторые проблемы с реализацией этого режима в данной среде (они будут описаны ниже),
поэтому основным агентом является шагающий агент.

Каждый агент имеет две очереди:
- Очередь действий
- Очередь постройки и удаления (в шагающем агенте не реализовано)

При инициализации агента производится построение очереди постройки, состоящее из объектов, в которые передаются контекст (координаты блока, цвет, действие - удалить или построить). Также в объект передается сам агент, над которым будет производиться управление.
Сами объекты представляют собой сущности класса BuildingQueueItem с методом process в котором производится построение очереди действий.

Действия представляют собой команды, поддерживаемые средой.

Когда происходит запрос действия от агента, он инициализируется, если до этого не был инициализирован,
затем, идет проверка на наличие элементов в очереди действий.
Если элементов нет, тогда производится проверка на наличие элемента из очереди построения,
и вызывает метод process этого элемента. Затем производится повторная проверка очереди действий
и возвращается действие из этой очереди, если она не пуста.
Когда элементов в обеих очередях нет, возвращается пропуск-действие и агент переходит в состояние is_done.

Расчет всех необходимых действий производится при инициализации агента. Разделение на очередь действий и постройки нужно для оптимизации оперативной памяти.

Для корректной работы необходимо передать в конструктор агента карту, которую необходимо получить (target_grid) и начальное положение агента.

## Ограничения агентов
- Агенты пока могу работать только из известной начальной позиции.
- Агенты будут корректно строить только при пустом начальном состоянии карты среды.

## Описание алгоритма шагающего агента

Сам алгоритм построения для шагающего агента очень прост. Построение ведется по столбцам сетки в зоне построения. Если в целевой сетке, на определенном столбце существует элемент на некоторой высоте, то агент строит "башню" с данной высотой в соответствующих координатах. Когда башня достроена, агент спрыгивает с башни, удаляя ненужные блоки. И так для каждого столбца - с лева на право, с севера на юг.

Исходный код агента можно посмотреть [здесь](https://github.com/v3code/IGLU-simple-agent/blob/master/core/builder_walking.py)

# Проблемы с летающим агентом
- Непрерывным является только движение камерой, движение агента идет с одинаковой скоростью при любом значении в управляющем векторе от -1 до 1 (кроме соответственно 0).
- Движение по вертикали вверх и вниз не проходит сквозь блоки
- 1 шаг в шагающем пространстве действий равен 0.25 длины блока, а в летающем - 0.75, из-за чего возникает смещение позиции при переходе к определенным координатам, и, соответственно, ошибках при построении и удалении блоков

## Возможные исправления
- Использовать движение по одной оси и управлять направлением при помощи камеры
- Использовать коррекцию камерой при построении и удалении блоков


# Оценка агента с использованием NLP моделью

In [ ]:
import os
import numpy as np

import gym
from gridworld.tasks import DUMMY_TASK

from core.builder_walking import BuilderWalking

from gridworld.data import IGLUDataset

from nlp_model.agent import GridPredictor, get_dialog
from nlp_model.utils import plot_voxel, compute_metric

dataset = IGLUDataset(dataset_version='v0.1.0-rc1', )


def eval_agent():
    grid_predictor = GridPredictor()
    env = gym.make('IGLUGridworld-v0', vector_state=True, render_size=(800, 600))
    # print(np.array(task.starting_grid).nonzero())
    env.set_task(DUMMY_TASK)
    total_score = []

    for j, (task_id, session_id, subtask_id, subtask) in enumerate(dataset):
        str_id = str(task_id) + '-session-' + str(session_id).zfill(3) + '-subtask-' + str(subtask_id).zfill(3)
        print('Starting task:', str_id)

        dialog = get_dialog(subtask)
        predicted_grid = grid_predictor.predict_grid(dialog)

        obs = env.reset()
        agent = BuilderWalking(predicted_grid, obs['agentPos'][:3])
        done = False

        while not agent.is_done:
            action = agent.get_action()

            obs, reward, done, info = env.step(action)

        if not os.path.exists('plots'):
            os.makedirs('plots')

        f1_score = round(compute_metric(obs['grid'], subtask.target_grid)['completion_rate_f1'], 3)
        results = {'F1': f1_score}
        total_score.append(f1_score)
        results_str = " ".join([f"{metric}: {value}" for metric, value in results.items()])
        plot_voxel(predicted_grid, text=str_id + ' ' + f'({results_str})' + "\n" + dialog).savefig(
            f'./plots/{str_id}-predicted.png')
        plot_voxel(subtask.target_grid, text=str_id + " (Ground truth)\n" + dialog).savefig(
            f'./plots/{str_id}-gt.png')

    print('Total F1 score:', np.mean(total_score))
eval_agent()

Loading parsed dataset failed. Downloading full dataset.


parsing dataset: 100%|██████████| 143/143 [00:03<00:00, 46.03it/s]


caching tasks dataset... done
